<a href="https://colab.research.google.com/github/hanseul1215/ESAA_study/blob/master/0523_recommend2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#파이썬 추천 시스템 패키지 - Surprise





In [1]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 4.4 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633732 sha256=a79c785862aa3d4fc344dca470dea860ac2da61e022cdfb49f5057b143e550d2
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


## Surprise 장점
- 다양한 추천 알고리즘(ex. 사용자 또는 아이템 기반 최근접 이웃 협업 필터링, SVD 등등)을 쉽게 적용해 추천 시스템을 구축할 수 있음
- 사이킷런의 핵심 API와 유사한 API명으로 작성됨

## Surprise를 이용한 추천 시스템 구축
- 사용자 아이디, 아이템 아이디, 평점 데이터가 row 레벨 형태로 되어 있는 포맷의 데이터만 처리함

In [2]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy
from surprise.model_selection import train_test_split

In [5]:
from numpy.random.mtrand import random_integers
data = Dataset.load_builtin('ml-100k', prompt=False) # prompt=False
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [6]:
algo = SVD()
algo.fit(trainset)

In [7]:
predictions = algo.test(testset)
print('prediction type:', type(predictions), ' size:', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type: <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.72010193867742, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.6775986490847434, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.023855573075837, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.3353374714759934, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.436501889524071, details={'was_impossible': False})]

was_impossible = True 면 예측값을 생성할 수 없는 데이터라는 의미

In [8]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.72010193867742),
 ('882', '291', 3.6775986490847434),
 ('535', '507', 4.023855573075837)]

In [9]:
#사용자 아이디, 아이템 아이디는 문자열로 입력
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 3.93   {'was_impossible': False}


In [10]:
accuracy.rmse(predictions)

RMSE: 0.9481


0.9480532454865005

# Surprise 주요 모듈 소개
- Dataset.load_builtin : 무비렌즈 데이터 로딩
- Dataset.load_from_file : OS파일에서 데이터 로딩
- Dataset.load_from_df : 판다스 df에서 데이터를 로딩

In [12]:
# OS파일 데이터를 Surprise 데이터 세트로 로딩
import pandas as pd
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/recommend/ratings.csv')
#ratings_noh.csv 파일로 언로드 시 인덱스와 헤더를 모두 제거한 새로운 파일 생성
ratings.to_csv('/content/drive/MyDrive/Colab Notebooks/recommend/ratings_noh.csv', index=False, header=False)

In [13]:
from surprise import Reader
reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('/content/drive/MyDrive/Colab Notebooks/recommend/ratings_noh.csv',reader=reader)

Reader 클래스 파라미터  
- line_format(칼럼을 순서대로 나열)  
- sep (칼럼을 분리하는 분리자)
- rating_scale (평점 값의 최소~최대 평점을 설정)

In [14]:
trainset, testset = train_test_split(data, test_size = .25, random_state=0)
algo = SVD(n_factors = 50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8908


0.8907754769926038

In [15]:
# 판다스 df 데이터를 Surprise 데이터 세트로 로딩
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/recommend/ratings.csv')
reader = Reader(rating_scale = (0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size = .25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predidctions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8908


0.8907754769926038

## Surprise 추천 알고리즘 클래스

- SVD : 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
- KNNBasic : 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
- BaselineOnly : 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘


## 베이스라인 평점
- 개인의 성향을 반영해 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것
- 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 
> 사용자 편향 점수: 사용자별 아이템 평점 평균 값 - 전체 평균 평점  
> 아이템 편향 점수: 아이템별 평점 평균 값 - 전체 평균 평점

In [16]:
# 교차 검증과 하이퍼 파라미터 튜닝
from surprise.model_selection import cross_validate

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/recommend/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8859  0.8911  0.9013  0.9051  0.9008  0.8968  0.0072  
MAE (testset)     0.6849  0.6869  0.6948  0.6951  0.6941  0.6911  0.0043  
Fit time          5.59    5.14    6.34    5.95    5.16    5.64    0.46    
Test time         0.29    0.14    0.15    0.14    0.14    0.17    0.06    


{'fit_time': (5.588737726211548,
  5.141812801361084,
  6.3373448848724365,
  5.950109481811523,
  5.158848762512207),
 'test_mae': array([0.68490764, 0.68690538, 0.69475719, 0.69508774, 0.69408505]),
 'test_rmse': array([0.88592582, 0.89105978, 0.90134116, 0.90508219, 0.90077319]),
 'test_time': (0.2890286445617676,
  0.14028573036193848,
  0.14560461044311523,
  0.14409470558166504,
  0.14396214485168457)}

In [17]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

gs = GridSearchCV(SVD, param_grid, measures = ['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8990889478399516
{'n_epochs': 20, 'n_factors': 50}


In [18]:
# Surprise를 이용한 개인화 영화 추천 시스템 구축
## rating.csv 데이터를 학습과 테스트 데이터로 분리하지 않고 전체를 학습 데이터로 사용
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data_folds = DatasetAutoFolds(ratings_file = '/content/drive/MyDrive/Colab Notebooks/recommend/ratings_noh.csv', reader = reader)

trainset = data_folds.build_full_trainset()

In [19]:
algo = SVD(n_epochs = 20, n_factors = 50, random_state = 0)
algo.fit(trainset)

In [26]:
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/recommend/movies.csv')
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
41       42  Dead Presidents (1995)  Action|Crime|Drama


In [27]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 2.64   {'was_impossible': False}


In [28]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()
    total_movies = movies['movieId'].tolist()

    #모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))
    
    return unseen_movies

In [29]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 45 추천 대상 영화 수: 62378 전체 영화 수: 62423


In [30]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    #알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행 수 결과를 list로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    #predictions list 객체는 surprise의 Predictions 객체를 원소로 가짐
    #이를 est 값으로 정렬하기 위해 아래의 sortkey_est 함수 정의
    #sortkey_est 함수는 list 객체의 sort() 함수의 키 값을 사용되어 정렬 수행
    def sortkey_est(pred):
        return pred.est

    #sortkey_esr() 반환값의 내림차순으로 정렬 수행 후 top_n개의 최상위 값 추출
    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions=predictions[:top_n]

    #top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

    top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

In [31]:
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

평점 매긴 영화 수: 45 추천 대상 영화 수: 62378 전체 영화 수: 62423


In [32]:
print('#### Top-10 추천 영화 리스트 ####')
for top_movie in top_movie_preds:
    print(top_movie[1], ':', round(top_movie[2], 3))

#### Top-10 추천 영화 리스트 ####
Usual Suspects, The (1995) : 4.401
Godfather, The (1972) : 4.371
It Happened One Night (1934) : 4.364
Casablanca (1942) : 4.336
Maltese Falcon, The (1941) : 4.321
All About Eve (1950) : 4.309
It's a Wonderful Life (1946) : 4.304
African Queen, The (1951) : 4.29
Cinema Paradiso (Nuovo cinema Paradiso) (1989) : 4.284
Modern Times (1936) : 4.272
